In [1]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import getpass
import os

/Users/litalertlumprasert/opt/anaconda3/lib/python3.9/site-packages/snowflake/connector/options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (7.0.0), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
con = snowflake.connector.connect(
    user='LITA',
    password=getpass.getpass(prompt='Password: ', stream=None),
    account='tga60420',
    warehouse='COMPUTE_WH',
    database='ANALYST',
    schema='VEHICLE_REGISTRATIONS'
)

Password: ········


In [3]:
ls data/

DECKED_NEW_VEHICLE_DATA_202211.csv*  archive/
DECKED_USED_VEHICLE_DATA_202211.csv*


In [4]:
files = [x for x in os.listdir('./data/') if ((x[0] != '~') & (('DECKED_') in x) & ('.zip' not in x))]

In [5]:
files

['DECKED_NEW_VEHICLE_DATA_202211.csv', 'DECKED_USED_VEHICLE_DATA_202211.csv']

In [6]:
for i, file in enumerate(files):
    print(i, file)
    if i == 0:
        df = pd.read_csv(f'data/{file}')
        if 'NEW' in file:
            print('adding new')
            df['NEW_USED'] = 'NEW'
        elif 'USED' in file:
            print('adding used')
            df['NEW_USED'] = 'USED'
    else:
        df_i = pd.read_csv(f'data/{file}')
        if 'NEW' in file:
            print('adding new')            
            df_i['NEW_USED'] = 'NEW'
        elif 'USED' in file:
            print('adding used')
            df_i['NEW_USED'] = 'USED'            
        df = pd.concat([df, df_i])

0 DECKED_NEW_VEHICLE_DATA_202211.csv
adding new
1 DECKED_USED_VEHICLE_DATA_202211.csv
adding used


In [7]:
df['NEW_USED'].value_counts()

NEW     3123015
USED    2662381
Name: NEW_USED, dtype: int64

In [8]:
df.columns

Index(['REPORT_YEAR_MONTH', 'BODY_STYLE', 'CYLINDERS', 'DOORS', 'DRIVE_TYPE',
       'FUEL_TYPE', 'GVW_CODE', 'GVW_DESCRIPTION', 'MAKE', 'MODEL', 'ORIGIN',
       'SEGMENT', 'SERIES', 'SUB_SERIES', 'BED_SIZE', 'CAB_SIZE', 'TURBO',
       'MODEL_YEAR', 'REGISTRATION_TYPE', 'DELAER_NAME', 'DEALER_ADDRESS',
       'DEALER_TOWN', 'DEALER_STATE', 'DEALER_ASSIGNED_ZIP', 'DEALER_DMA',
       'DEALER_COUNTY', 'OCCUPATION', 'VEHICLE_COUNT', 'NEW_USED'],
      dtype='object')

In [9]:
df.loc[df['REGISTRATION_TYPE'] == '(01) TOTAL (INCLUDES 2 11 18)' , 'VEHICLE_COUNT'].sum()

8671181

In [10]:
df['VEHICLE_COUNT'].sum() - df.loc[df['REGISTRATION_TYPE'] == '(01) TOTAL (INCLUDES 2 11 18)' , 'VEHICLE_COUNT'].sum()

2725338

In [11]:
df_ads = df[~df['REGISTRATION_TYPE'].str.contains('INCLUDES')]

In [12]:
df_ads['REGISTRATION_TYPE'].value_counts()

(13) COMMERCIAL                                 76281
(12) RENTAL/LEASE                               28615
(17) GOVERNMENT                                 20005
(15) PARTICIPATING INDEPENDENT LEASE - FLEET    17122
(14) MANUFACTURER SPONSORED LEASE - FLEET        1684
Name: REGISTRATION_TYPE, dtype: int64

In [13]:
query = 'SELECT MAX(REPORT_YEAR_MONTH) FROM ANALYST.VEHICLE_REGISTRATIONS.REGISTRATIONS'

In [14]:
cur = con.cursor()
cur.execute(query)

In [15]:
max_date = cur.fetchone()[0]

In [16]:
max_date

202210

In [17]:
df['REPORT_YEAR_MONTH'].value_counts()

202205    556834
202203    555262
202204    544224
202207    543477
202201    537722
202208    526534
202202    519067
202206    517245
202210    516857
202209    501734
202211    466440
Name: REPORT_YEAR_MONTH, dtype: int64

In [18]:
df_new = df[df['REPORT_YEAR_MONTH'] > max_date]

In [19]:
df_new['REPORT_YEAR_MONTH'].value_counts()

202211    466440
Name: REPORT_YEAR_MONTH, dtype: int64

In [20]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466440 entries, 0 to 197530
Data columns (total 29 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   REPORT_YEAR_MONTH    466440 non-null  int64  
 1   BODY_STYLE           466440 non-null  object 
 2   CYLINDERS            462860 non-null  object 
 3   DOORS                466214 non-null  object 
 4   DRIVE_TYPE           466440 non-null  object 
 5   FUEL_TYPE            466440 non-null  object 
 6   GVW_CODE             466440 non-null  int64  
 7   GVW_DESCRIPTION      466440 non-null  object 
 8   MAKE                 466440 non-null  object 
 9   MODEL                466440 non-null  object 
 10  ORIGIN               466440 non-null  object 
 11  SEGMENT              466440 non-null  object 
 12  SERIES               466440 non-null  object 
 13  SUB_SERIES           466440 non-null  object 
 14  BED_SIZE             466088 non-null  object 
 15  CAB_SIZE         

In [21]:
df_new.head()

,REPORT_YEAR_MONTH,BODY_STYLE,CYLINDERS,DOORS,DRIVE_TYPE,FUEL_TYPE,GVW_CODE,GVW_DESCRIPTION,MAKE,MODEL,...,DELAER_NAME,DEALER_ADDRESS,DEALER_TOWN,DEALER_STATE,DEALER_ASSIGNED_ZIP,DEALER_DMA,DEALER_COUNTY,OCCUPATION,VEHICLE_COUNT,NEW_USED
0,202211,PICKUPS,4 CYLINDERS,2 DOORS,4X4 OR REAR WHEEL DRIVE,GASOLINE,1,0 - 6000 lbs.,TOYOTA,TACOMA,...,ACTON TOYOTA OF LITTLETON,221 GREAT RD,LITTLETON,MA,1460.0,BOSTON (MANCHESTER),MIDDLESEX,NOCODE,1,NEW
1,202211,PICKUPS,4 CYLINDERS,2 DOORS,4X4 OR REAR WHEEL DRIVE,GASOLINE,1,0 - 6000 lbs.,TOYOTA,TACOMA,...,AL HENDRICKSON TOYOTA,5201 W SAMPLE RD,COCONUT CREEK,FL,33073.0,MIAMI FT LAUDERDALE,BROWARD,UNKNOWN,2,NEW
2,202211,PICKUPS,4 CYLINDERS,2 DOORS,4X4 OR REAR WHEEL DRIVE,GASOLINE,1,0 - 6000 lbs.,TOYOTA,TACOMA,...,ANTWERPENS TOYOTA,12420 AUTO DR,CLARKSVILLE,MD,21029.0,BALTIMORE,HOWARD,NOCODE,2,NEW
3,202211,PICKUPS,4 CYLINDERS,2 DOORS,4X4 OR REAR WHEEL DRIVE,GASOLINE,1,0 - 6000 lbs.,TOYOTA,TACOMA,...,ATLANTA TOYOTA,2345 PLEASANT HILL RD,DULUTH,GA,30096.0,ATLANTA,GWINNETT,UNKNOWN,1,NEW
4,202211,PICKUPS,4 CYLINDERS,2 DOORS,4X4 OR REAR WHEEL DRIVE,GASOLINE,1,0 - 6000 lbs.,TOYOTA,TACOMA,...,AUTONATION TOYOTA FORT MYERS,2555 COLONIAL BLVD,FORT MYERS,FL,33907.0,FT MYERS NAPLES,LEE,UNKNOWN,1,NEW


In [22]:
df_new['SERIES'] = df_new['SERIES'].astype(str)

/var/folders/zp/mrm17z_n1lvb4lhxxv1jmzdh0000gp/T/ipykernel_14487/2089189252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['SERIES'] = df_new['SERIES'].astype(str)


In [23]:
write_pandas(con, df_new, 'REGISTRATIONS')

(True,
 1,
 466440,
 [('gkofw/file0.txt', 'LOADED', 466440, 466440, 1, 0, None, None, None, None)])